# Wikipedia RAG‌ Fusion

## Abstract class of QueryMaker and OpenAIQuery

In [15]:
from openai import OpenAI
from abc import ABC, abstractmethod


class QueryMaker(ABC):
    @abstractmethod
    def __init__(self):
        """
        Base class for making queries to external APIs.
        """
        pass

    @abstractmethod
    def query(self, prompt, **kwargs):
        """
        Method to be implemented by subclasses for making queries.

        :param prompt: The input prompt for the query.
        :param kwargs: Additional parameters for the query.
        :return: The query result.
        """
        pass

class OpenAIQuery(QueryMaker):
    def __init__(self, api_key):
        """
        Initialize the OpenAIQuery with an API key.

        :param api_key: Your OpenAI API key.
        """
        super().__init__()
        self.api_key = api_key
        self.client = OpenAI(api_key=self.api_key)

    def __call__(self, prompt, model="text-davinci-003", max_tokens=100, temperature=0.7):
        """
        Perform a single query to the OpenAI API with the given prompt.

        :param prompt: The input prompt to send to the OpenAI API.
        :param model: The model to use for the query (default is "text-davinci-003").
        :param max_tokens: The maximum number of tokens to include in the output (default is 100).
        :param temperature: Sampling temperature to control randomness (default is 0.7).
        :return: The API response as a string.
        """
        try:
            completion = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant"},
                    {"role": "user", "content": prompt}
                ]
            )

            return completion.choices[0].message.content
        except Exception as e:
            return f"An error occurred: {e}"
    def query(self, prompt, **kwargs):
        raise Exception("Not implemented method, to sending query calling object directly.")


## Document Porcessor

In [16]:
class DocumentProcessor:
    def __init__(self, processor):
        """
        Initialize the DocumentProcessor with a processing object.

        :param processor: An object with a callable interface for processing queries with the documents.
        """
        self.processor = processor

    def process(self, document, query):
        """
        Process a query string using the processor and the fixed set of documents.

        :param query: The query string to be processed.
        :return: The result of processing the query with the documents.
        """
        if not callable(self.processor):
            raise TypeError("The processor object must be callable.")

        prequote = 'با توجه به متن زیر به این سوال پاسخ بده:'
        postquote = 'فقط با توجه به این متن و بدون استفاده از اطلاعات خودت به این سوال پاسخ بده:'
        input_prompt = prequote + query + document + postquote + query
        return self.processor(input_prompt)


## Document Retriver

In [17]:
import requests

class DocumentRetriever:
    def __init__(self):
        """
        Initializes the DocumentRetriever instance.
        This class is designed to retrieve relevant Wikipedia pages and their content based on a given sentence.
        """
        self.wikipedia_api_url = "https://fa.wikipedia.org/w/api.php"

    def search_wikipedia(self, query):
        """
        Searches Wikipedia for pages related to the query.

        Args:
            query (str): The input query to search for.

        Returns:
            list: A list of page titles relevant to the query.
        """
        params = {
            'action': 'query',
            'format': 'json',
            'list': 'search',
            'srsearch': query,
            'utf8': 1,
        }

        response = requests.get(self.wikipedia_api_url, params=params)

        if response.status_code != 200:
            raise Exception(f"Error searching Wikipedia: {response.status_code}")

        search_results = response.json()
        titles = [item.get('title', "") for item in search_results.get('query', {}).get('search', [])]

        return titles

    def get_page_content(self, title):
        """
        Retrieves the content of a Wikipedia page by its title.

        Args:
            title (str): The title of the Wikipedia page.

        Returns:
            str: The content of the page.
        """
        params = {
            'action': 'query',
            'format': 'json',
            'prop': 'extracts',
            'explaintext': 1,
            'titles': title,
        }

        response = requests.get(self.wikipedia_api_url, params=params)

        if response.status_code != 200:
            raise Exception(f"Error fetching page content: {response.status_code}")

        page_data = response.json()
        pages = page_data.get('query', {}).get('pages', {})

        for page_id, page_info in pages.items():
            if page_id != "-1":  # Ensure the page exists
                return page_info.get('extract', "")

        return None
    
    def retrieve(self, sentence):
        """
        Takes a sentence, retrieves relevant Wikipedia pages, and returns their content.

        Args:
            sentence (str): The input sentence to search for relevant Wikipedia pages.

        Returns:
            dict: A dictionary where keys are Wikipedia page titles and values are their content.
        """
        if not isinstance(sentence, str):
            raise ValueError("Input must be a string.")

        # Search for relevant Wikipedia pages
        titles = self.search_wikipedia(sentence)
        documents = []

        for title in titles:
            page_content = self.get_page_content(title)
            if page_content:
                documents += [page_content]

        return documents


## Document Pipeline 

In [18]:
class DocumentProcessingPipeline:
    def __init__(self, retriever, processors):
        """
        Initializes the DocumentProcessingPipeline with a retriever and a list of processors.

        :param retriever: An instance of DocumentRetriever.
        :param processors: A list of DocumentProcessor instances.
        """
        self.retriever = retriever
        self.processors = processors

    def load_documents(self, query):
        self.documents = self.retriever.retrieve(query)

        # Ensure the number of processors is sufficient
        if len(self.processors) < len(self.documents):
            print("Not enough processors for the number of documents.")
            self.documents = self.documents[:len(self.processors)]

    def get_updated_documents(self, documents, results, query):
        new_text = '\n'.join(results)
        prefix = 'اطلاعات جدید هم در آخر بهت میدم که خلاصه ای از متون دیگر است. میتوانی از آنها هم استفاده کنی. \n'
        prefix += '\n این هم خلاضه یافته‌ها از متون دیکر است. از این جوابها استفاده کن برای کامل کردن پاسخ به سوال: \n' + query + 'اشکال ندارد اگر خلاصه ها نقص دارد. آن قسمتهایی را بگو که جواب مشخص است .'
        for i in range(len(documents)):
            documents[i] = prefix + new_text + documents[i]
        return documents
            
    def execute(self, query, num_runs=1):
        """
        Executes the pipeline: retrieves documents, processes them, and returns results.

        :return: List of processed results.
        """
        docs = self.documents + []
        
        from joblib import Parallel, delayed
        import joblib

        for i in range(num_runs):
            with joblib.parallel_backend("threading"):
                results = Parallel(n_jobs=len(self.processors))(delayed(processor.process)(doc, query) for doc, processor in zip(docs, self.processors))
                print('جوابها در این مرحله:')
                _ = [print(r) for r in results]
                docs = self.get_updated_documents(docs, results, query)

        return results

## In Action

In [19]:
from getpass import getpass
api_key = getpass()
document_processors = [DocumentProcessor(OpenAIQuery(api_key)) for i in range(10)]
retriever = DocumentRetriever()
pipe = DocumentProcessingPipeline(retriever, document_processors)

In [20]:
query = 'مساحت ایران چقدر است؟'

In [21]:
pipe.load_documents(query)
results = pipe.execute(query, num_runs=3)
'\n'.join(results)

جوابها در این مرحله:
مساحت ایران ۱٫۶۴ میلیون کیلومتر مربع است.
مساحت ایران حدود ۱٬۸۷۳٬۹۵۹ کیلومتر مربع تخمین زده شده است. این عدد شامل خاک و آب‌های سرزمینی ایران بوده و بر اساس محاسبات جدید و دقیق به دست آمده است. همچنین باید توجه داشت که مساحت ایران بدون احتساب آب‌های بین‌المللی و منطقه انحصاری اقتصادی به حدود ۱٬۶۳۲٬۲۱۰ کیلومتر مربع می‌رسد.
متن ارائه شده هیچ اطلاعاتی در مورد مساحت ایران ندارد. لذا نمی‌توان به سوال شما پاسخ داد.
متن ارائه شده به طور خاص به مساحت ایران اشاره‌ای ندارد. بنابراین، برای پاسخ به این سؤال باید به اطلاعات دیگری مراجعه کرد. مساحت ایران حدود ۱٬۶۴۸٬۱۹۵ کیلومتر مربع است.
متأسفانه متن ارائه شده اطلاعاتی در مورد مساحت ایران ندارد. بنابراین نمی‌توان به سوال شما پاسخ دقیقی داد. اگر سوال دیگری دارید، خوشحال می‌شوم که کمک کنم.
متن ارائه شده هیچ اشاره‌ای به مساحت ایران نکرده است. بنابراین، نمی‌توان پاسخ دقیقی به این سوال داد.
متن ارائه شده هیچ اطلاعاتی در مورد مساحت ایران ندارد. بنابراین، نمی‌توان بر اساس این متن به سوال شما پاسخ داد. اگر سوال دیگری دارید یا به اطلاعات خ

'مساحت ایران به طور کلی حدود ۱٫۶۴ میلیون کیلومتر مربع تخمین زده شده است. بر اساس برخی منابع، مساحت ایران با احتساب خاک و آب\u200cهای سرزمینی به حدود ۱٬۸۷۳٬۹۵۹ کیلومتر مربع می\u200cرسد. همچنین، اگر فقط خاک ایران بدون احتساب آب\u200cهای بین\u200cالمللی و منطقه انحصاری اقتصادی محاسبه شود، این عدد به حدود ۱٬۶۳۲٬۲۱۰ کیلومتر مربع خواهد رسید. بنابراین، می\u200cتوان گفت که مساحت ایران بسته به روش محاسبه و منبع، بین ۱٫۶۴ تا ۱٫۸۷ میلیون کیلومتر مربع متغیر است.\nمساحت ایران به طور کلی حدود ۱٫۶۴ میلیون کیلومتر مربع تخمین زده شده است. همچنین بر اساس برخی محاسبات جدید، مساحت ایران که شامل خاک و آب\u200cهای سرزمینی می\u200cشود، حدود ۱٬۸۷۳٬۹۵۹ کیلومتر مربع اعلام شده است. بدون احتساب آب\u200cهای بین\u200cالمللی و منطقه انحصاری اقتصادی، مساحت خاک ایران به حدود ۱٬۶۳۲٬۲۱۰ کیلومتر مربع می\u200cرسد. بنابراین می\u200cتوان گفت که مساحت ایران بسته به منبع و روش محاسبه، بین ۱٫۶۴ تا ۱٫۸۷ میلیون کیلومتر مربع تخمین زده می\u200cشود.\nمساحت ایران بسته به منابع مختلف متفاوت است. به طور کلی، مساحت ایران حدود ۱٫۶۴ میلی

In [22]:
print(results[-1])

مساحت ایران به طور تقریبی بین ۱٫۶۴ میلیون کیلومتر مربع تا ۱٬۸۷۳٬۹۵۹ کیلومتر مربع تخمین زده می‌شود. عدد دقیق‌تر که گاهی بیان می‌شود حدود ۱٬۶۳۲٬۲۱۰ کیلومتر مربع است. بنابراین، می‌توان گفت که مساحت ایران حدود ۱٫۶۴ میلیون کیلومتر مربع است و بسته به منابع مختلف این عدد ممکن است متفاوت باشد.
